**Это мое решение с кастомным обучением аугментации и имитации в ЧБ Изображений/Видео для хакатона
Использованные технологии: Yolov8s(для быстроты), кастомный датасет дефектоскопии + своя жесткая аугментация + Florence-2 чтобы валидировать предсказания Yolov8s**

##Данное решение было полностью создано мной за 48 часа в период хакатона

***Датасет не был предоставлен поэтому я работал со своим***

In [ ]:
import os
os.kill(os.getpid(), 9) # самый надежный способ перезапустить среду в Colab

In [1]:
!pip install --upgrade pip
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 # Убедитесь, что версия CUDA соответствует вашей среде Colab
!pip install flash-attn --no-build-isolation

!pip install --upgrade transformers
!pip install --upgrade huggingface-hub
!pip install accelerate
!pip install Pillow

# установим зависимости для самой модели.
# !pip install -e .  # Если вы клонировали репозиторий Florence-2
!pip install huggingface-hub==0.20.0
!pip install transformers==4.49.0
!pip install pyngrok
!pip install ultralytics


Looking in indexes: https://download.pytorch.org/whl/cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 23.4 MB/s  0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.1
    Uninstalling huggingface-hub-1.0.1:
      Successfully uninstalled huggingface-hub-1.0.1
  Using cached huggingface_hub-1.0.1-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-1.0.1-py3-none-any.whl (503 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.1 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 1.0.1 which is incompatible.
--- Установка завершена ---
Для применения изменений, пожалу

In [3]:
from pyngrok import ngrok

#@markdown **Введите снизу свой токен Ngrok для Gradio**
NGROK_TOKEN = ''#@param {type:"string"}
# Установка токена
ngrok.set_auth_token(NGROK_TOKEN)

ModuleNotFoundError: No module named 'pyngrok'

In [1]:
import gradio as gr
from ultralytics import YOLO
import cv2
import numpy as np
import tempfile
from PIL import Image, ImageEnhance, ImageFilter
import torch
from transformers import AutoProcessor, AutoModelForCausalLM
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
# @markdown #..Вставьте путь для своих весов yolo..
YOLO_MODEL_PATH = '' # @param {type:"string"}
# Дефолтный промпт
DEFAULT_CHECK_PROMPT_EN = (
    "Analyze this steel defect. Select the most appropriate class from: "
    "Crazing, Inclusion, Patches, Pitted Surface, Rolled-in Scale, or Scratches. "
    "If the class is Scratches, describe its depth and length. "
    "Respond with only the selected class and a brief justification for the choice."
)

# Загрузка моделей
try:
    model = YOLO(YOLO_MODEL_PATH)
    logger.info(f"Модель YOLO загружена из {YOLO_MODEL_PATH}")
except Exception as e:
    logger.error(f"Не удалось загрузить YOLO: {e}. Проверьте путь: {YOLO_MODEL_PATH}")
    model = None

# (ленивая загрузка флоренц 2)
florence_model = None
florence_processor = None

def load_florence():
    global florence_model, florence_processor
    if florence_model is None:
        try:
            logger.info("Загрузка Florence-2...")
            device = "cuda" if torch.cuda.is_available() else "cpu"
            dtype = torch.float16 if device == "cuda" else torch.float32

            florence_processor = AutoProcessor.from_pretrained(
                "microsoft/Florence-2-base",
                trust_remote_code=True
            )
            florence_model = AutoModelForCausalLM.from_pretrained(
                "microsoft/Florence-2-base",
                torch_dtype=dtype,
                trust_remote_code=True,
                attn_implementation="sdpa",
                use_flash_attention_2=False
            ).to(device)

            logger.info(f"Florence-2 загружена на {device} (режим SDPA) с dtype={dtype}")
        except Exception as e:
            logger.warning(f"Ошибка загрузки Florence-2 с SDPA: {e}. Пробуем базовый режим...")
            try:
                device = "cuda" if torch.cuda.is_available() else "cpu"
                dtype = torch.float32
                florence_processor = AutoProcessor.from_pretrained(
                    "microsoft/Florence-2-base",
                    trust_remote_code=True
                )
                florence_model = AutoModelForCausalLM.from_pretrained(
                    "microsoft/Florence-2-base",
                    torch_dtype=dtype,
                    trust_remote_code=True,
                    _attn_implementation="eager"
                ).to(device)
                logger.info(f"Florence-2 загружена на {device} (базовый режим 'eager') с dtype={dtype}")
            except Exception as e2:
                logger.error(f"Критическая ошибка загрузки Florence-2: {e2}")
                raise

    return florence_model, florence_processor

# --- Функции предобработки (без изменений) ---

def _apply_enhancements(gray_pil_image):
    """Применяет CLAHE, фильтры и улучшение к Ч/Б PIL изображению"""
    img_array = np.array(gray_pil_image)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(img_array)
    denoised = cv2.medianBlur(enhanced, 3)
    kernel = np.ones((3, 3), np.uint8)
    morphed = cv2.morphologyEx(denoised, cv2.MORPH_OPEN, kernel, iterations=1)
    pil_processed = Image.fromarray(morphed)
    sharpened = pil_processed.filter(ImageFilter.SHARPEN)
    enhancer_contrast = ImageEnhance.Contrast(sharpened)
    contrast_enhanced = enhancer_contrast.enhance(1.2)
    enhancer_brightness = ImageEnhance.Brightness(contrast_enhanced)
    final_img_pil = enhancer_brightness.enhance(1.1)
    return final_img_pil

def _base_resize_pad(image, target_size=(640, 640)):
    """Базовый ресайз и паддинг, возвращает PIL Image"""
    if isinstance(image, np.ndarray):
        pil_img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)) if len(image.shape) == 3 else Image.fromarray(image)
    else:
        pil_img = image

    if pil_img.mode != 'RGB':
        pil_img = pil_img.convert('RGB')

    pil_img.thumbnail(target_size, Image.Resampling.LANCZOS)
    new_img = Image.new("RGB", target_size, (128, 128, 128))
    paste_x = (target_size[0] - pil_img.width) // 2
    paste_y = (target_size[1] - pil_img.height) // 2
    new_img.paste(pil_img, (paste_x, paste_y))
    return new_img

def remove_shadows_and_highlights(gray_image_np):
    """Агрессивное удаление теней и бликов (Ч/Б numpy in -> Ч/Б numpy out)"""
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
    tophat = cv2.morphologyEx(gray_image_np, cv2.MORPH_TOPHAT, kernel)
    blackhat = cv2.morphologyEx(gray_image_np, cv2.MORPH_BLACKHAT, kernel)
    result = cv2.add(gray_image_np, tophat)
    result = cv2.subtract(result, blackhat)
    result = cv2.normalize(result, None, 0, 255, cv2.NORM_MINMAX)
    return result

def preprocess_image(image, target_size=(640, 640)):
    """Базовая предобработка (RGB numpy out)"""
    pil_img_padded = _base_resize_pad(image, target_size)
    gray_img = pil_img_padded.convert('L')
    enhanced_gray_pil = _apply_enhancements(gray_img)
    final_rgb = enhanced_gray_pil.convert('RGB')
    return np.array(final_rgb)

def advanced_preprocess(image, target_size=(640, 640)):
    """Продвинутая предобработка (RGB numpy out)"""
    pil_img_padded = _base_resize_pad(image, target_size)
    gray_img = pil_img_padded.convert('L')
    gray_np = np.array(gray_img)
    shadows_removed_np = remove_shadows_and_highlights(gray_np)
    shadows_removed_pil = Image.fromarray(shadows_removed_np)
    enhanced_gray_pil = _apply_enhancements(shadows_removed_pil)
    final_rgb = enhanced_gray_pil.convert('RGB')
    return np.array(final_rgb)

# --- /Функции предобработки ---


def analyze_defect_with_florence(crop_pil, yolo_class_name, custom_prompt=None):
    """
    Анализ дефекта с помощью Florence-2
    FIX: Полный отказ от post_process_generation, ручная обрезка токенов.
    """
    try:
        model, processor = load_florence()
        if model is None:
            return f"{yolo_class_name} (Ошибка: Florence-2 не загружена)"

        device = next(model.parameters()).device

        if crop_pil.width < 10 or crop_pil.height < 10:
            return f"{yolo_class_name} (crop слишком мал)"

        # 1. Выбираем промпт
        if custom_prompt and custom_prompt.strip() != "":
            task_prompt = custom_prompt
            logger.info(f"Используется кастомный промпт: {custom_prompt}")
        else:
            task_prompt = DEFAULT_CHECK_PROMPT_EN
            logger.info(f"Используется дефолтный промпт")

        # 2. Токенизируем промпт
        inputs = processor(
            text=task_prompt,
            images=crop_pil,
            return_tensors="pt"
        ).to(device)

        # --- FIX: (TypeError: float vs c10::Half) ---
        if device.type == 'cuda' and model.dtype == torch.float16:
            if inputs['pixel_values'].dtype == torch.float32:
                inputs['pixel_values'] = inputs['pixel_values'].to(torch.float16)
        # --- /FIX ---

        input_ids_length = inputs['input_ids'].shape[1]

        with torch.no_grad():
            generated_ids = model.generate(
                input_ids=inputs["input_ids"],
                pixel_values=inputs["pixel_values"],
                max_new_tokens=150, # Токены только для ОТВЕТА
                num_beams=3,
                do_sample=False,
                early_stopping=True
            )

        newly_generated_ids = generated_ids[:, input_ids_length:]

        # skip_special_tokens=True убирает <eos> и т.д.
        generated_text = processor.batch_decode(
            newly_generated_ids,
            skip_special_tokens=True
        )[0]

        # 7. Отказываемся от post_process_generation
        description = generated_text.strip()
        # --- / ГЛАВНЫЙ FIX ---

        if not description: # Если ответ пустой
            description = yolo_class_name

        if yolo_class_name.lower().replace("_", " ") not in description.lower():
             description = f"{yolo_class_name.title()}: {description}"

        return description

    except Exception as e:
        logger.error(f"Ошибка Florence-2 анализа: {e}")
        import traceback
        traceback.print_exc()
        # Возвращаем полную ошибку для отладки
        return f"{yolo_class_name} (Ошибка: {e})"

# Обработка изображений
def detect_image(image, use_advanced=True, use_florence=True,
                 custom_prompt=None,
                 progress=gr.Progress()):
    """
    Детекция дефектов на изображении с двухконтурным анализом
    """
    if image is None:
        return None, "Изображение не загружено", None, None
    if model is None:
        return None, "Ошибка: Модель YOLO не загружена. Проверьте путь.", None, None

    progress(0, desc="Инициализация...")

    progress(0.1, desc="Предобработка изображения...")
    if use_advanced:
        preprocessed = advanced_preprocess(image, target_size=(640, 640))
    else:
        preprocessed = preprocess_image(image, target_size=(640, 640))

    progress(0.3, desc="Детекция YOLOv8...")
    results = model.predict(source=preprocessed, save=False, conf=0.25, verbose=False)
    frame_with_boxes = results[0].plot()

    preds = []
    crops_gallery = []
    boxes_list = list(results[0].boxes)

    for idx, box in enumerate(boxes_list):
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        yolo_label = results[0].names[cls_id]

        x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
        crop_array = preprocessed[y1:y2, x1:x2]

        if crop_array.size == 0:
            continue

        crop_pil = Image.fromarray(crop_array)

        final_label = yolo_label
        if use_florence and crop_pil.width > 20 and crop_pil.height > 20:
            progress(0.3 + 0.7 * (idx / max(1, len(boxes_list))),
                     desc=f"Анализ дефекта {idx+1}/{len(boxes_list)}...")

            florence_description = analyze_defect_with_florence(
                crop_pil,
                yolo_label,
                custom_prompt
            )
            final_label = florence_description

        preds.append({
            "id": idx + 1,
            "yolo_class": yolo_label,
            "florence_description": final_label if use_florence else None,
            "confidence": conf,
            "bbox": [x1, y1, x2, y2]
        })

        crops_gallery.append((crop_pil, final_label))

    progress(1.0, desc="Готово!")

    if preds:
        output_text = f"## Результаты двухконтурного анализа\n\n**Обнаружено дефектов: {len(preds)}**\n\n### Детали детекции:"
        for pred in preds:
            output_text += f"\n\n**Дефект #{pred['id']}:**\n"
            output_text += f"- YOLO класс: **{pred['yolo_class']}**\n"
            output_text += f"- 🔍 Уверенность: **{pred['confidence']*100:.1f}%**\n"

            # --- FIX: Упрощенная логика для вывода ---
            # Показываем описание, если оно есть И оно не совпадает с классом YOLO
            if use_florence and pred['florence_description'] and pred['florence_description'] != pred['yolo_class']:
                desc = pred['florence_description']

                # Убираем префикс "Класс: ", если он есть
                if desc.startswith(f"{pred['yolo_class'].title()}: "):
                    desc = desc[len(pred['yolo_class'])+2:].strip()

                # Показываем очищенное описание (или ошибку)
                output_text += f"- Ответ Florence-2: _{desc}_"
            # --- /FIX ---

            output_text += f"\n- Координаты bbox: [{pred['bbox'][0]}, {pred['bbox'][1]}, {pred['bbox'][2]}, {pred['bbox'][3]}]"
    else:
        output_text = "**Дефектов не обнаружено**"

    output_text += f"""

---
- ✅ Размер: 640x640
- ✅ Предобработка: {'Продвинутая (тени/блики удалены)' if use_advanced else 'Базовая'}
- ✅ Контур 1 (YOLO): Детекция локализации ✓
- {'✅' if use_florence else '❌'} Контур 2 (Florence-2): {'Детальный анализ ✓' if use_florence else 'Отключен'}
"""

    return frame_with_boxes, output_text, preprocessed, crops_gallery if crops_gallery else None

# Обработка видео
def detect_video(video, use_advanced=False, use_florence=False,
                 custom_prompt=None,
                 progress=gr.Progress()):
    if video is None:
        return None, "Видео не загружено", None
    if model is None:
        return None, "Ошибка: Модель YOLO не загружена.", None

    temp_out = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4")
    cap = cv2.VideoCapture(video)

    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 20
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames == 0:
        return None, "Не удалось прочитать видео", None

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = None

    frame_count = 0
    total_detections = 0
    defect_counts = {}
    first_frame_processed = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        progress(frame_count / total_frames, desc=f"Кадр {frame_count}/{total_frames}")

        if use_advanced:
            processed_frame = advanced_preprocess(frame, target_size=(640, 640))
        else:
            processed_frame = preprocess_image(frame, target_size=(640, 640))

        if first_frame_processed is None:
            first_frame_processed = processed_frame.copy()

        results = model.predict(processed_frame, save=False, conf=0.25, verbose=False)
        frame_out = results[0].plot()

        for box in results[0].boxes:
            total_detections += 1
            cls_id = int(box.cls[0])
            label = results[0].names[cls_id]

            if use_florence: # ⚠️ Это все еще ОЧЕНЬ медленно
                x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
                crop = processed_frame[y1:y2, x1:x2]
                if crop.size > 0:
                    crop_pil = Image.fromarray(crop)
                    florence_desc = analyze_defect_with_florence(
                        crop_pil, label, custom_prompt
                    )
                    label = florence_desc

            defect_counts[label] = defect_counts.get(label, 0) + 1

        if out is None:
            h, w, _ = frame_out.shape
            out = cv2.VideoWriter(temp_out.name, fourcc, fps, (w, h))

        out.write(frame_out)

    cap.release()
    if out: out.release()

    stats_text = f"## Статистика обработки видео\n\n### Основные метрики:\n- 🎬 **Всего кадров:** {total_frames}\n- 🔍 **Всего детекций:** {total_detections}\n- 📈 **Среднее на кадр:** {total_detections/max(total_frames, 1):.2f}\n\n### Распределение дефектов:"

    if defect_counts and total_detections > 0:
        for defect, count in sorted(defect_counts.items(), key=lambda x: x[1], reverse=True)[:10]:
            percentage = (count / total_detections) * 100
            stats_text += f"\n- **{defect}:** {count} ({percentage:.1f}%)"
    else:
        stats_text += "\nДефектов не обнаружено"

    stats_text += f"\n\n---\n### Настройки обработки:\n- ✅ Предобработка: {'Продвинутая' if use_advanced else 'Базовая'}\n- {'✅' if use_florence else '❌'} Florence-2: {'Включен (медленнее)' if use_florence else 'Отключен'}"

    return temp_out.name, stats_text, first_frame_processed

# Интерфейс Gradio
with gr.Blocks(title="YOLOv8 + Florence-2 Steel Defect Detection", theme=gr.themes.Soft()) as demo:

    gr.Markdown("# 🔍 YOLOv8 + Florence-2: Двухконтурная система дефектоскопии\n## NEU-DET: Детекция дефектов стальных поверхностей")

    with gr.Tabs() as tabs:
        # Вкладка для изображений
        with gr.Tab("📸 Режим изображений", id=0):
            with gr.Row():
                with gr.Column(scale=1):
                    image_input = gr.Image(label="Входное изображение", type="numpy")

                    with gr.Accordion("Настройки", open=True):
                        use_advanced_img = gr.Checkbox(
                            value=True,
                            label="Продвинутая предобработка",
                            info="Удаление теней и бликов (рекомендуется)"
                        )
                        use_florence_img = gr.Checkbox(
                            value=True,
                            label="Использовать Florence-2",
                            info="Детальный анализ каждого дефекта"
                        )

                        custom_prompt_img = gr.Textbox(
                            label="Кастомный промпт (опционально)",
                            placeholder=f"Если пусто, используется дефолтный: '{DEFAULT_CHECK_PROMPT_EN[:50]}...'",
                            lines=3
                        )

                    image_button = gr.Button("Анализировать", variant="primary", size="lg")

                with gr.Column(scale=1):
                    image_output = gr.Image(label="Результат с аннотациями", type="numpy")
                    image_stats = gr.Markdown("### Результаты появятся после обработки")

            with gr.Row():
                with gr.Column():
                    image_preprocessed = gr.Image(
                        label="После предобработки",
                        # FIX: Удален 'info'
                        type="numpy"
                    )

                with gr.Column():
                    image_crops = gr.Gallery(
                        label="Вырезанные дефекты",
                        columns=3,
                        rows=2,
                        object_fit="contain"
                    )

            image_button.click(
                fn=detect_image,
                inputs=[image_input, use_advanced_img, use_florence_img, custom_prompt_img],
                outputs=[image_output, image_stats, image_preprocessed, image_crops]
            )

        # Вкладка для видео
        with gr.Tab("🎬 Режим видео", id=1):
            gr.Markdown("### Обработка видео\n⚠️ Florence-2 для видео очень медленный! Рекомендуется отключить.")

            with gr.Row():
                with gr.Column(scale=1):
                    video_input = gr.Video(label="Входное видео")

                    with gr.Accordion("Настройки", open=True):
                        use_advanced_vid = gr.Checkbox(value=False, label="🔧 Продвинутая предобработка", info="Медленнее, но качественнее")
                        use_florence_vid = gr.Checkbox(value=False, label="🤖 Использовать Florence-2", info="⚠️ ОЧЕНЬ медленно! ~1 сек на дефект")
                        custom_prompt_vid = gr.Textbox(
                            label="Кастомный промпт (опционально)",
                            placeholder=f"Если пусто, используется дефолтный: '{DEFAULT_CHECK_PROMPT_EN[:50]}...'",
                            lines=3
                        )

                    video_button = gr.Button("Обработать", variant="primary", size="lg")

                with gr.Column(scale=1):
                    video_output = gr.Video(label="Результат")
                    video_stats = gr.Markdown("### Статистика")

            with gr.Row():
                video_preview = gr.Image(label="Первый кадр (предобработка)", type="numpy")

            video_button.click(
                fn=detect_video,
                inputs=[video_input, use_advanced_vid, use_florence_vid, custom_prompt_vid],
                outputs=[video_output, video_stats, video_preview]
            )

    # Информация о системе
    with gr.Accordion("О двухконтурной системе", open=False):
        gr.Markdown(f"""
        ### Архитектура
        **Контур 1: YOLOv8 (NEU-DET)**
        - Задача: Локализация (где дефект?)

        **Контур 2: Florence-2-base**
        - Задача: Понимание (что за дефект?)

        ### Классы NEU-DET:
        1. Crazing (Cr), 2. Inclusion (In), 3. Patches (Pa),
        4. Pitted Surface (PS), 5. Rolled-in Scale (RS), 6. Scratches (Sc)
        """)

    with gr.Accordion("Советы по использованию", open=False):
        gr.Markdown(f"""
        ### Рекомендации:

        **Для изображений:**
        - ✅ Включите Florence-2 для детального анализа
        - ✅ Используйте кастомные промпты для специфичных задач

        **Для видео:**
        - ⚠️ Отключите Florence-2 (очень медленно)

        **Промпты:**
        - Оставьте поле пустым для дефолтного анализа (классификация + обоснование).
        - **Дефолтный промпт (автоматически):** `{DEFAULT_CHECK_PROMPT_EN}`
        - **Пример кастомного:** "Опиши тип, цвет и серьезность дефекта"
        """)

# Запуск
if __name__ == "__main__":
    if model is None:
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print("!!! ВНИМАНИЕ: Модель YOLO не загружена.                 !!!")
        print(f"!!! Проверьте путь в переменной YOLO_MODEL_PATH: {YOLO_MODEL_PATH} !!!")
        print("!!! Демо запустится, но не будет работать.           !!!")
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

    # Загружаем Florence-2 при старте, чтобы первый запуск был быстрее
    logger.info("Попытка предварительной загрузки Florence-2...")
    load_florence()

    demo.launch(share=True, debug=True)



SyntaxError: invalid syntax (ipython-input-2253580511.py, line 14)

In [19]:
# 3. Функция для выполнения задачи
def run_example(task_prompt, image, text_input=None):
    """
    Выполняет заданный промпт с изображением.
    """
    if image.mode != "RGB":
        image = image.convert("RGB")

    # --- ИСПРАВЛЕНИЕ: Объединяем task_prompt и text_input в один аргумент 'text' ---

    # 1. Объединение промпта задачи и дополнительного текстового ввода
    full_prompt = task_prompt
    if text_input:
        full_prompt += text_input
    else:
        full_prompt = f"{task_prompt}\nCaption:"
    # 2. Формирование входов для модели (Используем объединенный текст 'full_prompt')
    inputs = processor(
        text=full_prompt,
        images=image,
        return_tensors="pt"
    ).to(device)

    # -----------------------------------------------------------------------------

    # Генерация ответа
    generated_ids = model.generate(
        **inputs,
        max_new_tokens=1024,
        eos_token_id=processor.tokenizer.eos_token_id,
        do_sample=False,
        num_beams=3 # Рекомендованное значение
    )

    # Декодирование сгенерированного текста
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

    # Парсинг ответа
    parsed_text = processor.post_process_generation(
        generated_text,
        task_prompt=task_prompt,
        image_size=(image.width, image.height)
    )
    return parsed_text

In [25]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor
from PIL import Image
import requests
from io import BytesIO

# 1. Определение устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_DTYPE = torch.bfloat16
print(f"Используемое устройство: {device}")
print(f"Используемый тип данных (DType): {MODEL_DTYPE}")

# 2. Загрузка модели и процессора
model_id = "microsoft/Florence-2-base"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=MODEL_DTYPE,
    trust_remote_code=True
).to(device)

processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
print("Модель и процессор Florence-2 успешно загружены.")

# 3. Функция для выполнения задачи
# 3. Функция для выполнения задачи
def run_example(task_prompt, image, text_input=None, desired_new=48):
    if image.mode != "RGB":
        image = image.convert("RGB")

    # 1. Формирование промпта (ИСПРАВЛЕНО!)
    # Для Florence-2 нужно передавать только токен,
    # и только для VQA использовать формат Q/A

    if task_prompt.strip().upper() == "<VQA>" and text_input:
        # Стандартный формат VQA
        full_prompt = f"{task_prompt}\nQ: {text_input}\nA:"
        # Префикс для парсинга (в случае VQA)
        generation_prefix = "A:"
    else:
        # Для всех остальных задач (CAPTION, OD и т.д.)
        # Процессор должен видеть только токен задачи.
        full_prompt = task_prompt.strip()
        generation_prefix = "" # Не ищем префикс

    # 2. Формируем inputs
    inputs = processor(text=full_prompt, images=image, return_tensors="pt")

    # Переносим на устройство
    for k, v in inputs.items():
        if torch.is_floating_point(v):
            inputs[k] = v.to(MODEL_DTYPE).to(device)
        else:
            inputs[k] = v.to(device)

    # ... (Оставляем gen_kwargs и max_length как есть)
    gen_kwargs = {
        "input_ids": inputs["input_ids"],
        "pixel_values": inputs["pixel_values"],
        "attention_mask": inputs["attention_mask"],
    }
    prompt_len = inputs["input_ids"].shape[1]
    max_length = prompt_len + desired_new
    pad_id = getattr(processor.tokenizer, "pad_token_id", None) or processor.tokenizer.eos_token_id
    eos_id = processor.tokenizer.eos_token_id

    # Генерация
    generated = model.generate(
        **gen_kwargs,
        max_length=max_length,
        do_sample=False,
        num_beams=3,
        pad_token_id=pad_id,
        eos_token_id=eos_id,
        early_stopping=True,
        return_dict_in_generate=True
    )

    seq = generated.sequences[0]
    decoded = processor.tokenizer.decode(seq, skip_special_tokens=True)

    # 3. Извлечение ответа (на основе VQA-префикса)
    if generation_prefix and generation_prefix in decoded:
        # Если это VQA, разделяем по "A:"
        parsed_text = decoded.split(generation_prefix, 1)[-1].strip()
    else:
        # Для остальных задач (CAPTION) - просто удаляем промпт из начала
        parsed_text = decoded[len(full_prompt):].strip() if decoded.startswith(full_prompt) else decoded.strip()
        # Для Captioning нужно найти то, что сгенерировано после токена
        if task_prompt.strip().upper() == "<CAPTION>":
             # Часто модель начинает сгенерированный текст сразу после токена
             # поэтому просто возвращаем декодированный текст (так как skip_special_tokens=True)
             parsed_text = decoded.strip()

    return parsed_text

# 4. Пример использования
image_url = "https://thumb.cloud.mail.ru/weblink/thumb/xw1/uqhp/7mBK8QqGo/IMG_6873.JPG"
response = requests.get(image_url)
image = Image.open(BytesIO(response.content))
print("\nТестовое изображение загружено.")

prompt_vqa = "<DETAILED_CAPTION>"
#question = "Where is the cat?"
result_vqa = run_example(prompt_vqa, image, text_input=None, desired_new=48)

print("\n--- Результат VQA ---")
print(f"Ответ Florence-2: {result_vqa}")


Используемое устройство: cuda
Используемый тип данных (DType): torch.bfloat16
Модель и процессор Florence-2 успешно загружены.

Тестовое изображение загружено.

--- Результат VQA ---
Ответ Florence-2: The image shows a man standing on top of a stage, holding a microphone in his hand. Behind him is a wall with a curtain draped over it.
